In [1]:
import re
import requests
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
categories = ['https://www.amazon.com/Best-Sellers-Appliances/zgbs/appliances/ref=zg_bs_nav_0',
              'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/ref=zg_bs_nav_0',
              'https://www.amazon.com/best-sellers-books-Amazon/zgbs/books/ref=zg_bs_nav_0'
             ]


In [4]:
first_url = categories[0]

In [5]:
driver.get(first_url)

In [6]:
body_el = driver.find_element(By.CSS_SELECTOR, "body")
html_str = body_el.get_attribute('innerHTML')

In [7]:
html_obj = HTML(html=html_str)

In [8]:
page_links = [f'https://www.amazon.com{x}' for x in html_obj.links if x.startswith('/')]
# new_links = [x for x in new_links if 'product-reviews/' not in x]

In [9]:
# page_links

In [10]:
def scrape_product_page(url, title_lookup = '#productTitle', price_lookup = '.a-offscreen'):
    driver.get(url)
    time.sleep(.5)
    body_el = driver.find_element(By.CSS_SELECTOR, "body")
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [11]:
# first_product_link = page_links[0]
# first_product_link

In [12]:
# https://www.amazon.com/FRIGIDAIRE-EFIC189-Silver-Compact-Silver-Packaging/dp/B075QP7SCP/
# https://www.amazon.com/Silonn-Countertop-Self-Cleaning-Compact-SLIM02/dp/B09N8T9F8J/

# <base-url>/<slug>/dp/<product_id>/

In [13]:
# my_regex_pattern = r'https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/'
# my_url ='https://www.amazon.com/Upstreman-Adjustable-Thermostat-Refrigerator-Black-BR321/dp/B09RWFZTWW/'

In [14]:
# regex = re.compile(my_regex_pattern)

In [15]:
# my_match = regex.match(my_url)
# print(my_match)
# my_match['product_id']

In [16]:
# my_match['slug']

In [17]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [18]:
# page_links = [x for x in page_links if extract_product_id_from_url(x) != None]

def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links

cleaned_links = clean_page_links(page_links)

In [19]:
len(page_links) #== len(cleaned_links)

168

In [20]:
len(cleaned_links)

29

In [21]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {'url': link, 'product_id': product_id, 'title': title, 'price': price}
        data_extracted.append(product_data)
    return data_extracted

In [22]:
extracted_data = perform_scrape(cleaned_items=cleaned_links)

https://www.amazon.com/Broan-403001-Capable-Under-Cabinet-30-Inch/dp/B000KKMFB2/ref=zg_bs_appliances_sccl_19/140-5995169-8446846?pd_rd_i=B0B1JCXN1N&psc=1 Broan-NuTone 30-inch Under-Cabinet Convertible Range Hood with 2-Speed Exhaust Fan and Light, MAX 210 CFM, White $62.00
https://www.amazon.com/Silonn-Countertop-Portable-Self-cleaning-Bullet-shaped/dp/B08ZYJ8CRX/ref=zg_bs_appliances_sccl_1/140-5995169-8446846?pd_rd_i=B09BW71ZHK&psc=1 Silonn Ice Makers Countertop, 9 Cubes Ready in 6 Mins, 26lbs in 24Hrs, Self-Cleaning Ice Machine with Ice Scoop and Basket, 2 Sizes of Bullet Ice for Home Kitchen Office Bar Party $105.99
https://www.amazon.com/FRIGIDAIRE-EFIC189-Silver-Compact-Silver-Packaging/dp/B075QP7SCP/ref=zg_bs_appliances_sccl_8/140-5995169-8446846?pd_rd_i=B075QP7SCP&psc=1 FRIGIDAIRE EFIC189-Silver Compact Ice Maker, 26 lb per Day, Silver (Packaging May Vary) $124.90
https://www.amazon.com/SPACEKEEPER-Furniture-Extendable-Refrigerators-Dishwashers/dp/B09JFLQT9C/ref=zg_bs_appliances

https://www.amazon.com/Thereye-Countertop-Portable-Self-Cleaning-Stainless/dp/B09MVV717N/ref=zg_bs_appliances_sccl_29/140-5995169-8446846?pd_rd_i=B09MVV717N&psc=1 Thereye Countertop Nugget Ice Maker, Pebble Ice Maker Machine, 30lbs Per Day, 2 Ways Water Refill, 3Qt Water Reservoir & Self-Cleaning, Stainless Steel Finish Ice Machine for Home Office Bar Party $339.99
https://www.amazon.com/GE-Profile-Countertop-Nugget-Maker/dp/B07YF9SGBW/ref=zg_bs_appliances_sccl_2/140-5995169-8446846?pd_rd_i=B09YXFTWJS&psc=1 GE Profile Opal | Countertop Nugget Ice Maker with Side Tank | Portable Ice Machine with Bluetooth Connectivity | Smart Home Kitchen Essentials | Stainless Steel Finish | Up to 24 lbs. of Ice Per Day GE Profile Opal 2.0 | Countertop Nugget Ice Maker with Side Tank | Ice Machine with WiFi Connectivity | Smart Home Kitchen Essentials | Stainless Steel
https://www.amazon.com/AGLUCKY-Machine-Countertop-Portable-Makers/dp/B08FZYMWJT/ref=zg_bs_appliances_sccl_3/140-5995169-8446846?pd_rd_i

In [23]:
print(extracted_data)

[{'url': 'https://www.amazon.com/Broan-403001-Capable-Under-Cabinet-30-Inch/dp/B000KKMFB2/ref=zg_bs_appliances_sccl_19/140-5995169-8446846?pd_rd_i=B0B1JCXN1N&psc=1', 'product_id': 'B000KKMFB2', 'title': 'Broan-NuTone 30-inch Under-Cabinet Convertible Range Hood with 2-Speed Exhaust Fan and Light, MAX 210 CFM, White', 'price': '$62.00'}, {'url': 'https://www.amazon.com/Silonn-Countertop-Portable-Self-cleaning-Bullet-shaped/dp/B08ZYJ8CRX/ref=zg_bs_appliances_sccl_1/140-5995169-8446846?pd_rd_i=B09BW71ZHK&psc=1', 'product_id': 'B08ZYJ8CRX', 'title': 'Silonn Ice Makers Countertop, 9 Cubes Ready in 6 Mins, 26lbs in 24Hrs, Self-Cleaning Ice Machine with Ice Scoop and Basket, 2 Sizes of Bullet Ice for Home Kitchen Office Bar Party', 'price': '$105.99'}, {'url': 'https://www.amazon.com/FRIGIDAIRE-EFIC189-Silver-Compact-Silver-Packaging/dp/B075QP7SCP/ref=zg_bs_appliances_sccl_8/140-5995169-8446846?pd_rd_i=B075QP7SCP&psc=1', 'product_id': 'B075QP7SCP', 'title': 'FRIGIDAIRE EFIC189-Silver Compact 